In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 填充和步幅

**在所有侧边填充1个像素**

In [3]:
// def comp_conv2d(conv2d, X):
//     X = X.reshape((1, 1) + X.shape)
//     Y = conv2d(X)
//     return Y.reshape(Y.shape[2:])

// conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
// X = torch.rand(size=(8, 8))
// comp_conv2d(conv2d, X).shape

In [4]:
torch::Tensor comp_conv2d(torch::nn::Sequential conv2d, torch::Tensor X)
{
    X = torch::constant_pad_nd(X, at::IntArrayRef({0, 1, 0, 1}));
    auto Y = conv2d->forward(X);
    return Y.reshape(at::IntArrayRef({Y.size(2), Y.size(3)}));
}

In [5]:
torch::nn::Sequential conv2d{torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 1, {3, 3}).stride(1).padding(1).bias(false))};
auto X = torch::rand({8, 8});
X = X.reshape({1,1,8,8});
auto Z = comp_conv2d(conv2d, X);
printT(Z.sizes());

Z.sizes() = 
[9, 9]
<<--->>



**填充不同的高度和宽度**

In [6]:
torch::nn::Sequential conv2d_1{torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 1, {5, 3}).stride(1).padding({2, 1}).bias(false))};
Z = comp_conv2d(conv2d_1, X);
printT(Z.sizes());

Z.sizes() = 
[9, 9]
<<--->>



**将高度和宽度的步幅设置为2**

In [7]:
torch::nn::Sequential conv2d_2{torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 1, {3, 3}).stride({2, 2}).padding(1).bias(false))};
Z = comp_conv2d(conv2d_2, X);
printT(Z.sizes());

Z.sizes() = 
[5, 5]
<<--->>



**一个稍微复杂的例子**

In [8]:
// conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
torch::nn::Sequential conv2d_2{torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 1, {3, 5}).stride({3, 4}).padding({0, 1}).bias(false))};
Z = comp_conv2d(conv2d_2, X);
printT(Z.sizes());

Z.sizes() = 
[3, 2]
<<--->>

